<a href="https://colab.research.google.com/github/YusukeTakahashi2001/Pytorch-Tutorial_Des2020/blob/main/1203_CNN_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')
%cd drive/'My Drive'/FashionMnist-Practice

Mounted at /content/drive
/content/drive/My Drive/FashionMnist-Practice


In [ ]:
!pip install comet_ml
# import comet_ml at the top of your file
from comet_ml import Experiment

# import comet_ml at the top of your file
from comet_ml import Experiment

# Create an experiment with your api key:
experiment = Experiment(
    api_key="1H4oWzrtQdGj3uKZIDjQomMQs",
    project_name="fashionmnist-practice",
    workspace="yusuketakahashi2001",
)

     |████████████████████████████████| 245kB 8.4MB/s 
     |████████████████████████████████| 204kB 13.7MB/s 
     |████████████████████████████████| 512kB 14.7MB/s 
  Created wheel for configobj: filename=configobj-5.0.6-cp36-none-any.whl size=34547 sha256=3fe5cc2aeeee7a2d36bfb9a6da7096ddd9df27506beff5c8fa2a89ec334951fc
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e0b35e2660695be7219a2d351ee0
Successfully built configobj


COMET INFO: Experiment is live on comet.ml https://www.comet.ml/yusuketakahashi2001/fashionmnist-practice/28db3b1c0d8645cc9c6326c138b05b00



In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models,transforms,datasets
import torch.nn.functional as F

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
class CNNNet(nn.Module):
    def __init__(self,in_channel):
        super(CNNNet,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=in_channel,out_channels=16,kernel_size=3,stride=1),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1),
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=1),
            # out torch.Size([10, 128, 20, 20])
        )
        self.dence = nn.Sequential(
            nn.Linear(128*4*4,128),
            nn.Dropout(p=0.2),
            nn.Linear(128,64),
            nn.Dropout(p=0.2),
            nn.Linear(64,10),  
        )
    def forward(self,x):
        x = F.relu(self.layer1(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.layer2(x))
        x = F.max_pool2d(x,2,2)
        x = x.view(x.size()[0], -1)
        x = self.dence(x)
        
        return x
        
    def check_net_size(self,x):
        x = self.layer1(x)
        out = self.layer2(x)
        
        return out

In [ ]:
def h_cal(H,P,S,F):
  out = ((H + 2*P -F)/S) + 1
  return out

In [ ]:
print(h_cal(28,0,1,3),h_cal(26,0,1,3),h_cal(28,0,1,3),h_cal(12,0,1,3),h_cal(10,0,1,3))

26.0 24.0 26.0 10.0 8.0


In [ ]:
hyper_params = {
    "batch_size": 10,
    "num_epochs": 4,
    "learning_rate": 0.01,
    "in_channel":1
   
}

In [ ]:
class ImageTransform():
    def __init__(self):
        self.data_out = transforms.Compose([
            transforms.ToTensor()
        ])
        
    def __call__(self,img):
        return self.data_out(img)

In [ ]:
def main():
    experiment.log_parameters(hyper_params)
    net = CNNNet(in_channel=hyper_params["in_channel"])
    net.to(device)
    optimizer = optim.Adam(net.parameters(),lr=hyper_params["learning_rate"])
    criterion = nn.CrossEntropyLoss()
    trans = ImageTransform()
    
    #dataload
    dtrain = torch.utils.data.DataLoader(
        datasets.FashionMNIST('./data/fashion-mnist',train=True,download=True,transform=trans),
        batch_size = hyper_params["batch_size"],
        shuffle = True
    )
    
    dtest = torch.utils.data.DataLoader(
        datasets.FashionMNIST('./data/fashion-mnist',train=False,download=True,transform=trans),
        batch_size = hyper_params["batch_size"],
        shuffle = False
    )
    
    loss_train = []
    loss_test = []
    acc_test = []

    for epoch in range(1,hyper_params["num_epochs"]+1):
        batch_loss_train = []
        batch_loss_test = []
        correct = 0
        total = 0
        
        #train
        net.train()
        for xbatch_train,tbatch_train in dtrain:
            xbatch_train = xbatch_train.to(device)
            tbatch_train = tbatch_train.to(device)
            #Initialize grad
            net.zero_grad()
            ybatch_train = net.forward(xbatch_train)
            lbatch_train = criterion(ybatch_train,tbatch_train)
            lbatch_train.backward()
            optimizer.step()
            batch_loss_train.append(lbatch_train.tolist())
            
        net.eval()
        for xbatch_test,tbatch_test in dtest:
            xbatch_test = xbatch_test.to(device)
            tbatch_test = tbatch_test.to(device)
            ybatch_test = net.forward(xbatch_test)
            lbatch_test = criterion(ybatch_test,tbatch_test)
            batch_loss_test.append(lbatch_test.tolist())
            
            batch_label_test = ybatch_test.cpu().argmax(1).numpy()
            tbatch_test = tbatch_test.cpu().numpy()
            correct += np.sum((batch_label_test - tbatch_test) == 0)
            total +=1
            
        loss_train.append(np.mean(batch_loss_train))
        loss_test.append(np.mean(batch_loss_test))
        acc_test_batch = correct/total
        acc_test.append(acc_test_batch)
        
        print('EPOCH: {},TRAIN LOSS:{:.3f},TEST LOSS:{:.3f},TEST ACC:{:.3f}'.format(
            epoch,
            np.mean(loss_train),
            np.mean(loss_test),
            np.mean(acc_test)
        ))


    experiment.end()
    
    
        
            
        
            
        
        
    
    
    

In [ ]:
if __name__ == '__main__':
  main()

EPOCH: 1,TRAIN LOSS:2.320,TEST LOSS:2.311,TEST ACC:100.000
EPOCH: 2,TRAIN LOSS:2.321,TEST LOSS:2.331,TEST ACC:100.000
EPOCH: 3,TRAIN LOSS:2.322,TEST LOSS:2.326,TEST ACC:100.000


COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/yusuketakahashi2001/fashionmnist-practice/28db3b1c0d8645cc9c6326c138b05b00
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [7800] : (0.5005528330802917, 4.67109489440918)
COMET INFO:   Uploads:
COMET INFO:     code                : 1 (22 KB)
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     model graph         : 1
COMET INFO:     notebook            : 1
COMET INFO:     os packages         : 1
COMET INFO: ---------------------------


EPOCH: 4,TRAIN LOSS:2.322,TEST LOSS:2.326,TEST ACC:100.000


COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
